In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import patches
%matplotlib inline
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fonm
from matplotlib import font_manager, rc
mpl.rcParams['axes.unicode_minus'] = False
font_path = "C:/Windows/Fonts/HMKMRHD.ttf"
font = fonm.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [3]:
Basic_dir = os.path.dirname(os.path.dirname(os.path.realpath('EDA.ipynb')))
PATH = Basic_dir + '\\Data\\Data(PenW_B)'
os.listdir(PATH)

['sample_submission(0).csv',
 'sample_submission(1).csv',
 'sample_submission(2).csv',
 'sample_submission(3).csv',
 'sample_submission(3_statsmodel).csv',
 'sample_submission.csv',
 'test.csv',
 'test_Ch.csv',
 'train.csv',
 'train_Ch.csv',
 'train_pred_sex.csv']

In [11]:
train = pd.read_csv(PATH + '\\train_Ch.csv')
print(train.isna().sum().sum())
train.head()

0


,Species,Island,Clutch Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,Delta_15_N,Delta_13_C,Body_Mass
0,0,0,0,50.0,15.3,220,0,8.30515,-25.19017,5550
1,1,1,1,49.5,19.0,200,0,9.63074,-24.34684,3800
2,0,0,0,45.1,14.4,210,1,8.51951,-27.01854,4400
3,0,0,0,44.5,14.7,214,1,8.20106,-26.16524,4850
4,0,0,1,49.6,16.0,225,0,8.38324,-26.84272,5700


In [10]:
test = pd.read_csv(PATH + '\\test_Ch.csv')
print(test.isna().sum().sum())
test.head()

0


,id,Species,Island,Clutch Completion,Culmen_Length,Culmen_Depth,Flipper_Length,Sex,Delta_15_N,Delta_13_C
0,0,0,0,0,52.0,20.7,210.0,0,9.43146,-24.68440
1,1,1,1,0,55.9,17.0,228.0,0,8.31180,-26.35425
2,2,2,0,0,38.9,18.8,190.0,1,8.36936,-26.11199
3,3,0,0,0,45.2,16.6,191.0,1,9.62357,-24.78984
4,4,2,1,1,37.9,18.6,172.0,1,8.38404,-25.19837


In [70]:
from sklearn.linear_model import LinearRegression
x_list = ['Flipper_Length', 'Species', 'Sex', 'Delta_15_N'] # (2) - (2) > (1) / intercept = False
lm = LinearRegression(fit_intercept = True)
lm.fit(train[x_list], train.Body_Mass)
print(lm.coef_)

[  26.01750618 -199.41778763 -514.52791806 -321.74699639]


In [69]:
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

RMSE(lm.predict(train[x_list]), train.Body_Mass)

318.4856873299448

In [73]:
# 정답파일 작성
predict = pd.read_csv(PATH + '\\sample_submission.csv')
len(predict), len(test)
predict['Body Mass (g)'] = lm.predict(test[x_list])
predict.to_csv(PATH + '\\sample_submission(4_lR).csv', index = False)
pd.read_csv(PATH + '\\sample_submission(4_lR).csv').head()

,id,Body Mass (g)
0,0,4718.174555
1,1,5347.319120
2,2,3626.188423
3,3,3647.503204
4,4,3153.150066


In [17]:
from statsmodels.formula.api import ols 
from statsmodels.api import OLS, add_constant
x_list = x_list = ['Flipper_Length', 'Species', 'Sex', 'Delta_15_N']
formula = 'Body_Mass~' + '+'.join(x_list)
model = OLS.from_formula(formula, data = train[x_list + ['Body_Mass']])
result = model.fit()
# print(result.params)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Body_Mass   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     141.0
Date:                Tue, 28 Dec 2021   Prob (F-statistic):           3.82e-42
Time:                        16:32:31   Log-Likelihood:                -816.91
No. Observations:                 114   AIC:                             1644.
Df Residuals:                     109   BIC:                             1658.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2289.0422   1192.754      1.919      0.058     -74.958    4653.042
Flipper_Length    26.0175      4.022      6.468      0.000      18.046      33.989
Species         -199.4178     61.539     -3.241      0.002    -321.386     -77.449
Sex             -514.5279     71.654     -7.181      0.000    -656.544    -372.512
Delta_15_N      -321.7470     65.717     -4.896      0.000    -451.996    -191.498
==============================================================================
Omnibus:                        2.266   Durbin-Watson:                   2.092
Prob(Omnibus):                  0.322   Jarque-Bera (JB):                1.717
Skew:                           0.260   Prob(JB):                        0.424
Kurtosis:                       3.301   Cond. No.                     8.11e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.11e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true-pred)))
    return score

RMSE(result.predict(train[x_list]), train.Body_Mass)

313.2376110744562

In [41]:
# 릿지모델
from sklearn.linear_model import Ridge, Lasso

ridge = Ridge(alpha = 0.1, solver = 'sag', random_state = 7)
ridge.fit(train[x_list], train.Body_Mass)
RMSE(ridge.predict(train[x_list]), train.Body_Mass)

313.5242744851528

In [60]:
# 라쏘 모델
from sklearn.linear_model import Ridge, Lasso

lasso = Lasso(alpha = 1)
lasso.fit(train[x_list], train.Body_Mass)
RMSE(lasso.predict(train[x_list]), train.Body_Mass)

313.27093656258785

In [54]:
# SGDRegressor
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(max_iter=0.1,
                       penalty="l2",
                       random_state=42)
sgd.fit(train[x_list],train.Body_Mass)
RMSE(sgd.predict(train[x_list]), train.Body_Mass)

4397.28222982923

In [67]:
# 엘라스틱 넷

from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha = 0.01, l1_ratio = 0.5, random_state=42)
elastic_net.fit(train[x_list],train.Body_Mass)
RMSE(elastic_net.predict(train[x_list]), train.Body_Mass)

313.35147300670116